In [1]:
import requests
import json
import pandas as pd
import time
import os

In [2]:
root_url = 'http://www.nomisweb.co.uk'

# api_url = '/api/v01/dataset/NM_1_1/geography.def.sdmx.json' # list all possible geography sets
api_url = '/api/v01/dataset/NM_1_1/geography/2092957697.def.sdmx.json' # list of all geography sets in UK
# api_url = '/api/v01/dataset/NM_1_1/geography/2092957697TYPE441.def.sdmx.json' # major towns and cities within United Kingdom

url = root_url + api_url

response = requests.get(url)
json.loads(response.text)

{'structure': {'codelists': {'codelist': [{'agencyid': 'NOMIS',
     'id': 'CL_1_1_GEOGRAPHY',
     'code': [{'annotations': {'annotation': [{'annotationtext': 'countries',
          'annotationtitle': 'TypeName'},
         {'annotationtext': 499, 'annotationtitle': 'TypeCode'},
         {'annotationtext': 'K02000001', 'annotationtitle': 'GeogCode'}]},
       'description': {'value': 'United Kingdom', 'lang': 'en'},
       'value': 2092957697},
      {'annotations': {'annotation': [{'annotationtext': 'True',
          'annotationtitle': 'IsAbstractCode'},
         {'annotationtext': 1, 'annotationtitle': 'TypeCode'},
         {'annotationtext': '1991 frozen wards',
          'annotationtitle': 'TypeName'},
         {'annotationtext': 2092957697, 'annotationtitle': 'ParentCode'},
         {'annotationtext': 11330, 'annotationtitle': 'ChildCount'}]},
       'description': {'value': '1991 frozen wards within United Kingdom',
        'lang': 'en'},
       'value': '2092957697TYPE1'},
     

In [3]:
api_url = '/api/v01/dataset/NM_1_1/geography/2092957697TYPE441.def.sdmx.json' # major towns and cities within United Kingdom
url = root_url + api_url

response = requests.get(url)
cities_json = json.loads(response.text)
cities_json = cities_json['structure']['codelists']['codelist'][0]['code']
cities_list = []
for city in cities_json:
    geog_code = city['annotations']['annotation'][2]['annotationtext']
    name = city['description']['value']
    id = city['value']
    city_dict = {
        'id': id,
        'name': name,
        'geog_code': geog_code
    }
    cities_list.append(city_dict)
cities_df = pd.DataFrame(cities_list)
cities_df

,id,name,geog_code
0,1849688065,Barnsley,J01000001
1,1849688066,Basildon,J01000002
2,1849688067,Basingstoke,J01000003
3,1849688068,Bath,J01000004
4,1849688069,Bedford,J01000005
...,...,...,...
107,1849688172,Woking,J01000108
108,1849688173,Wolverhampton,J01000109
109,1849688174,Worcester,J01000110
110,1849688175,Worthing,J01000111


In [4]:
try:
    c2011_lookup = pd.read_csv(r'.\data\raw\geoportal\Lower_Layer_Super_Output_Area_(2011)_to_Major_Towns_and_Cities_(December_2015)_Lookup_in_England_and_Wales.csv')
except:
    os.chdir('..')
    c2011_lookup = pd.read_csv(r'.\data\raw\geoportal\Lower_Layer_Super_Output_Area_(2011)_to_Major_Towns_and_Cities_(December_2015)_Lookup_in_England_and_Wales.csv')
c2011_lookup = c2011_lookup.dropna(subset=['TCITY15NM'])
c2011_lookup

,LSOA11CD,LSOA11NM,TCITY15CD,TCITY15NM,FID
0,E01000001,City of London 001A,J01000055,London,1
1,E01000205,Barnet 035A,J01000055,London,2
2,E01000002,City of London 001B,J01000055,London,3
3,E01000003,City of London 001C,J01000055,London,4
4,E01000206,Barnet 033B,J01000055,London,5
...,...,...,...,...,...
34740,W01001703,Cardiff 045B,J01000020,Cardiff,34741
34742,W01001857,Cardiff 042C,J01000020,Cardiff,34743
34743,W01001858,Cardiff 034A,J01000020,Cardiff,34744
34744,W01001859,Cardiff 034B,J01000020,Cardiff,34745


In [5]:
c2021_lookup = pd.read_csv(r'.\data\raw\geoportal\LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales_(Version_2).csv')
c2021_lookup = c2021_lookup[['LSOA11CD', 'LSOA21CD']]
c2021_lookup = c2021_lookup.merge(c2011_lookup, on='LSOA11CD', how='inner')
c2021_lookup = c2021_lookup.drop('LSOA11CD', axis=1)
c2021_lookup = c2021_lookup.rename(columns={'LSOA11NM': 'LSOA21NM'})
c2021_lookup

,LSOA21CD,LSOA21NM,TCITY15CD,TCITY15NM,FID
0,E01000001,City of London 001A,J01000055,London,1
1,E01000002,City of London 001B,J01000055,London,3
2,E01000003,City of London 001C,J01000055,London,4
3,E01000005,City of London 001E,J01000055,London,7
4,E01000006,Barking and Dagenham 016A,J01000055,London,8
...,...,...,...,...,...
16037,W01001697,Cardiff 036D,J01000020,Cardiff,34735
16038,W01001698,Cardiff 036E,J01000020,Cardiff,34738
16039,W01001699,Cardiff 049A,J01000020,Cardiff,34739
16040,W01001702,Cardiff 045A,J01000020,Cardiff,34740


In [6]:
# api_url = '/api/v01/dataset/def.sdmx.json' # view all datasets
api_url = '/api/v01/dataset/def.sdmx.json?search=KS*'
# api_url = '/api/v01/dataset/NM_2021_1/def.sdmx.json'


url = root_url + api_url

response = requests.get(url)
json.loads(response.text)

{'structure': {'header': {'id': 'none',
   'prepared': '2024-04-15T22:34:40Z',
   'sender': {'contact': {'email': 'support@nomisweb.co.uk',
     'name': 'Nomis',
     'telephone': '+44(0) 191 3342680',
     'uri': 'https://www.nomisweb.co.uk'},
    'id': 'NOMIS'},
   'test': 'false'},
  'keyfamilies': {'keyfamily': [{'agencyid': 'NOMIS',
     'annotations': {'annotation': [{'annotationtext': 'Current (being actively updated)',
        'annotationtitle': 'Status'},
       {'annotationtext': 'Area (Hectares),Communal Establishment,Household,Household resident,Population density,Students,Schoolchildren,Usual Resident',
        'annotationtitle': 'Keywords'},
       {'annotationtext': 'Persons', 'annotationtitle': 'Units'},
       {'annotationtext': 'census_2011_ks',
        'annotationtitle': 'contenttype/sources'},
       {'annotationtext': 'oa,ps', 'annotationtitle': 'contenttype/geoglevel'},
       {'annotationtext': 2.2, 'annotationtitle': 'contenttype/censusrelease'},
       {'annota

In [7]:
# api_url = '/api/v01/dataset/def.sdmx.json?search=C2021TS*'
# url = root_url + api_url
# response = requests.get(url)

# c2021_datasets_json = json.loads(response.text)
# c2021_datasets_json = c2021_datasets_json['structure']['keyfamilies']['keyfamily']
# c2021_datasets_metadata_list = []
# for dataset in c2021_datasets_json:
#     annotations = dataset['annotations']['annotation']
#     for annotation in annotations:
#         if annotation['annotationtitle'] == 'contenttype/geoglevel':
#             geoglevel = annotation['annotationtext']
#         if annotation['annotationtitle'] == 'Mnemonic':
#             mnemonic = annotation['annotationtext']
#         if annotation['annotationtitle'] == 'MetadataText0':
#             description = annotation['annotationtext']
#     id = dataset['id']
#     metadata = {
#         'id': id,
#         'mnemonic': mnemonic,
#         'description': description,
#         'geoglevel': geoglevel
#     }
#     c2021_datasets_metadata_list.append(metadata)
# c2021_datasets_metadata_df = pd.DataFrame(c2021_datasets_metadata_list)
# c2021_datasets_metadata_df.to_csv(r'.\data\raw\api\lsoa\counts\c2021\metadata.csv')
# c2021_datasets_metadata_df

In [8]:
# record_limit = 25000
# offset = 0

# for dataset in c2021_datasets_metadata_list:
#     id = dataset['id']
#     mnemonic = dataset['mnemonic']
#     geoglevel = dataset['geoglevel'].split(',')
#     if 'oa' not in geoglevel:
#         continue

#     print("downloading", mnemonic)
#     api_url = f'/api/v01/dataset/{id}.data.csv?recordlimit=10'
#     url = root_url + api_url
#     try:
#         dataset_df = pd.read_csv(url)
#     except:
#         time.sleep(60)
#         dataset_df = pd.read_csv(url)
#     dataset_df.to_csv('test.csv')
#     try:
#         stat_name = dataset_df.filter(regex='^C2021.*NAME$', axis=1).columns[0].lower()
#     except:
#         try:
#             stat_name = dataset_df.filter(regex='^CELL.*NAME$', axis=1).columns[0].lower()
#         except:
#             stat_name = dataset_df.filter(regex='^C_.*NAME$', axis=1).columns[0].lower()

#     last_record = offset

#     for i in range(1000):
#         print(f"trying records {last_record} - {last_record + record_limit}")
#         api_url = f'/api/v01/dataset/{id}.data.csv?geography=TYPE151&measures=20100&select=date,geography_code,geography_type,geography_typecode,{stat_name},obs_value&recordlimit={record_limit}&recordoffset={last_record}'
#         url = root_url + api_url

#         print(url)

#         try:
#             batch_df = pd.read_csv(url)
#         except pd.errors.EmptyDataError:
#             print("empty dataset")
#             break
#         except:
#             time.sleep(60)
#             batch_df = pd.read_csv(url)

#         display(batch_df.head())

#         if len(batch_df) == 0:
#             break

#         dest_file = f'./data/raw/api/lsoa/counts/c2021/{mnemonic}_{i:02}.csv'
#         batch_df.to_csv(dest_file)
#         print(f"chunk {i} saved in {dest_file}")

#         last_record += record_limit

In [9]:
api_url = '/api/v01/dataset/def.sdmx.json?search=c2011ks*'
url = root_url + api_url
response = requests.get(url)
json.loads(response.text)

{'structure': {'header': {'id': 'none',
   'prepared': '2024-04-15T22:34:46Z',
   'sender': {'contact': {'email': 'support@nomisweb.co.uk',
     'name': 'Nomis',
     'telephone': '+44(0) 191 3342680',
     'uri': 'https://www.nomisweb.co.uk'},
    'id': 'NOMIS'},
   'test': 'false'},
  'keyfamilies': {'keyfamily': [{'agencyid': 'NOMIS',
     'annotations': {'annotation': [{'annotationtext': 'Current (being actively updated)',
        'annotationtitle': 'Status'},
       {'annotationtext': 'Area (Hectares),Communal Establishment,Household,Household resident,Population density,Students,Schoolchildren,Usual Resident',
        'annotationtitle': 'Keywords'},
       {'annotationtext': 'Persons', 'annotationtitle': 'Units'},
       {'annotationtext': 'census_2011_ks',
        'annotationtitle': 'contenttype/sources'},
       {'annotationtext': 'oa,ps', 'annotationtitle': 'contenttype/geoglevel'},
       {'annotationtext': 2.2, 'annotationtitle': 'contenttype/censusrelease'},
       {'annota

In [10]:
api_url = '/api/v01/dataset/def.sdmx.json?search=c2011ks*'
url = root_url + api_url
response = requests.get(url)

c2011_datasets_json = json.loads(response.text)
c2011_datasets_json = c2011_datasets_json['structure']['keyfamilies']['keyfamily']
c2011_datasets_metadata_list = []
for dataset in c2011_datasets_json:
    annotations = dataset['annotations']['annotation']
    for annotation in annotations:
        if annotation['annotationtitle'] == 'contenttype/geoglevel':
            geoglevel = annotation['annotationtext']
        if annotation['annotationtitle'] == 'Mnemonic':
            mnemonic = annotation['annotationtext']
        if annotation['annotationtitle'] == 'MetadataText0':
            description = annotation['annotationtext']
    id = dataset['id']
    metadata = {
        'id': id,
        'mnemonic': mnemonic,
        'description': description,
        'geoglevel': geoglevel
    }
    c2011_datasets_metadata_list.append(metadata)
c2011_datasets_metadata_df = pd.DataFrame(c2011_datasets_metadata_list)
c2011_datasets_metadata_df.to_csv('./data/raw/api/lsoa/counts/c2011/metadata.csv')
c2011_datasets_metadata_df

,id,mnemonic,description,geoglevel
0,NM_144_1,c2011ks101ew,This table provides information that classifie...,"oa,ps"
1,NM_145_1,c2011ks102ew,This table provides information about the age ...,"oa,ps"
2,NM_146_1,c2011ks403ew,This table provides information about the room...,"oa,ps"
3,NM_158_1,c2011ks101uk,This table provides information that classifie...,"oa,ps"
4,NM_159_1,c2011ks102uk,This table provides information about the age ...,"oa,ps"
5,NM_603_1,c2011ks103ew,This table provides information that classifie...,"oa,ps"
6,NM_604_1,c2011ks104ew,This table provides information that classifie...,"oa,ps"
7,NM_605_1,c2011ks105ew,This table provides information that classifie...,"oa,ps"
8,NM_606_1,c2011ks106ew,This table provides information that classifie...,"oa,ps"
9,NM_607_1,c2011ks107ew,This table provides information about lone par...,"oa,ps"


In [11]:
record_limit = 25000
offset = 0

for dataset in c2011_datasets_metadata_list:
    id = dataset['id']
    mnemonic = dataset['mnemonic']
    geoglevel = dataset['geoglevel'].split(',')
    if 'oa' not in geoglevel:
        continue

    print("downloading", mnemonic)
    api_url = f'/api/v01/dataset/{id}.data.csv?recordlimit=10'
    url = root_url + api_url
    try:
        dataset_df = pd.read_csv(url)
    except:
        time.sleep(60)
        dataset_df = pd.read_csv(url)
    dataset_df.to_csv('test.csv')
    try:
        stat_name = dataset_df.filter(regex='^C2011.*NAME$', axis=1).columns[0].lower()
    except:
        try:
            stat_name = dataset_df.filter(regex='^CELL.*NAME$', axis=1).columns[0].lower()
        except:
            try:
                stat_name = dataset_df.filter(regex='^C_.*NAME$', axis=1).columns[0].lower()
            except:
                stat_name = dataset_df.filter(regex='RURAL_URBAN_TYPE', axis=1).columns[0].lower()

    last_record = offset

    for i in range(1000):
        print(f"trying records {last_record} - {last_record + record_limit}")
        api_url = f'/api/v01/dataset/{id}.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,{stat_name},obs_value&recordlimit={record_limit}&recordoffset={last_record}'
        url = root_url + api_url

        print(url)

        try:
            batch_df = pd.read_csv(url)
        except pd.errors.EmptyDataError:
            print("empty dataset")
            break
        except:
            time.sleep(60)
            batch_df = pd.read_csv(url)

        display(batch_df.head())

        if len(batch_df) == 0:
            break

        dest_file = f'./data/raw/api/lsoa/counts/c2011/{mnemonic}_{i:02}.csv'
        batch_df.to_csv(dest_file)
        print(f"chunk {i} saved in {dest_file}")

        last_record += record_limit

downloading c2011ks101ew
trying records 0 - 25000
http://www.nomisweb.co.uk/api/v01/dataset/NM_144_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=0


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2011,E01012334,2011 super output areas - lower layer,298,All usual residents,2466.0
1,2011,E01012334,2011 super output areas - lower layer,298,Males,1220.0
2,2011,E01012334,2011 super output areas - lower layer,298,Females,1246.0
3,2011,E01012334,2011 super output areas - lower layer,298,Lives in a household,2466.0
4,2011,E01012334,2011 super output areas - lower layer,298,Lives in a communal establishment,0.0


chunk 0 saved in ./data/raw/api/lsoa/counts/c2011/c2011ks101ew_00.csv
trying records 25000 - 50000
http://www.nomisweb.co.uk/api/v01/dataset/NM_144_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=25000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2011,E01020758,2011 super output areas - lower layer,298,All usual residents,0.0
1,2011,E01020758,2011 super output areas - lower layer,298,Males,0.0
2,2011,E01020758,2011 super output areas - lower layer,298,Females,0.0
3,2011,E01020758,2011 super output areas - lower layer,298,Lives in a household,0.0
4,2011,E01020758,2011 super output areas - lower layer,298,Lives in a communal establishment,0.0


chunk 1 saved in ./data/raw/api/lsoa/counts/c2011/c2011ks101ew_01.csv
trying records 50000 - 75000
http://www.nomisweb.co.uk/api/v01/dataset/NM_144_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=50000


...

chunk 85 saved in ./data/raw/api/lsoa/counts/c2011/c2011ks102ew_85.csv
trying records 2150000 - 2175000
http://www.nomisweb.co.uk/api/v01/dataset/NM_145_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=2150000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2011,E01011080,2011 super output areas - lower layer,298,Mean Age,NaN
1,2011,E01011080,2011 super output areas - lower layer,298,Median Age,0.5
2,2011,E01011080,2011 super output areas - lower layer,298,All usual residents,0.0
3,2011,E01011080,2011 super output areas - lower layer,298,Age 0 to 4,0.0
4,2011,E01011080,2011 super output areas - lower layer,298,Age 5 to 7,0.0


chunk 86 saved in ./data/raw/api/lsoa/counts/c2011/c2011ks102ew_86.csv
trying records 2175000 - 2200000
http://www.nomisweb.co.uk/api/v01/dataset/NM_145_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=2175000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2011,E01011706,2011 super output areas - lower layer,298,Age 65 to 74,0.0
1,2011,E01011706,2011 super output areas - lower layer,298,Age 75 to 84,0.0
2,2011,E01011706,2011 super output areas - lower layer,298,Age 85 to 89,0.0
3,2011,E01011706,2011 super output areas - lower layer,298,Age 90 and over,0.0
4,2011,E01011706,2011 super output areas - lower layer,298,Mean Age,NaN


chunk 87 saved in ./data/raw/api/lsoa/counts/c2011/c2011ks102ew_87.csv
trying records 2200000 - 2225000
http://www.nomisweb.co.uk/api/v01/dataset/NM_145_1.data.csv?geography=TYPE298&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=2200000


In [14]:
api_url = '/api/v01/dataset/def.sdmx.json?search=KS*'
url = root_url + api_url
response = requests.get(url)

c2001_datasets_json = json.loads(response.text)
c2001_datasets_json = c2001_datasets_json['structure']['keyfamilies']['keyfamily']
c2001_datasets_metadata_list = []
for dataset in c2001_datasets_json:
    annotations = dataset['annotations']['annotation']
    for annotation in annotations:
        if annotation['annotationtitle'] == 'contenttype/sources':
            source = annotation['annotationtext']
        if annotation['annotationtitle'] == 'contenttype/geoglevel':
            geoglevel = annotation['annotationtext']
        if annotation['annotationtitle'] == 'Mnemonic':
            mnemonic = annotation['annotationtext']
        if annotation['annotationtitle'] == 'MetadataText0':
            description = annotation['annotationtext']
    if source != 'census_2001_ks':
        continue
    id = dataset['id']
    metadata = {
        'id': id,
        'mnemonic': mnemonic,
        'description': description,
        'geoglevel': geoglevel,
        'source': source
    }
    c2001_datasets_metadata_list.append(metadata)
c2001_datasets_metadata_df = pd.DataFrame(c2001_datasets_metadata_list)
# c2001_datasets_metadata_df = c2001_datasets_metadata_df[c2001_datasets_metadata_df['source'] == 'census_2001_ks'].reset_index(drop=True)
# c2001_datasets_metadata_list = list(set(c2001_datasets_metadata_df['mnemonic']))
c2001_datasets_metadata_df.to_csv('./data/raw/api/msoa/c2001/metadata.csv')
c2001_datasets_metadata_df

,id,mnemonic,description,geoglevel,source
0,NM_1602_1,ks002,This dataset is about Age. It shows all peopl...,"oa,msoa,ward,la,region",census_2001_ks
1,NM_1603_1,ks003,This dataset is about Living Arrangements. It ...,"oa,msoa,ward,la,region",census_2001_ks
2,NM_1604_1,ks004,This dataset is about Marital Status. It shows...,"oa,msoa,ward,la,region",census_2001_ks
3,NM_1605_1,ks005,This dataset is about Country of Birth. It sho...,"oa,msoa,ward,la,region",census_2001_ks
4,NM_1606_1,ks006,This dataset is about Ethnic Group. This varia...,"oa,msoa,ward,la,region",census_2001_ks
5,NM_1607_1,ks007,This dataset is about Religion. It shows all p...,"oa,msoa,ward,la,region",census_2001_ks
6,NM_1608_1,ks008,This dataset is about Health and Provision of ...,"oa,msoa,ward,la,region",census_2001_ks
7,NM_1609_1,ks009a,This dataset is about Economic Activity. Infor...,"oa,msoa,ward,la,region",census_2001_ks
8,NM_1610_1,ks009b,This dataset is about Economic Activity. Infor...,"oa,msoa,ward,la,region",census_2001_ks
9,NM_1611_1,ks009c,This dataset is about Economic Activity. Infor...,"oa,msoa,ward,la,region",census_2001_ks


In [20]:
record_limit = 25000
offset = 0

for dataset in c2001_datasets_metadata_list[29:]:
    id = dataset['id']
    mnemonic = dataset['mnemonic']
    geoglevel = dataset['geoglevel'].split(',')
    if 'msoa' not in geoglevel:
        continue

    print("downloading", mnemonic)
    api_url = f'/api/v01/dataset/{id}.data.csv?recordlimit=10'
    url = root_url + api_url
    try:
        dataset_df = pd.read_csv(url)
    except:
        time.sleep(60)
        dataset_df = pd.read_csv(url)
    dataset_df.to_csv('test.csv')
    # try:
    stat_name = dataset_df.filter(regex='^(C2001|CELL|C_|COB_|DEPCHILD_|COMMTYPE_).*NAME$', axis=1).columns[0].lower()
    # except:
    #     try:
    #         stat_name = dataset_df.filter(regex='^CELL.*NAME$', axis=1).columns[0].lower()
    #     except:
    #         try:
    #             stat_name = dataset_df.filter(regex='^C_.*NAME$', axis=1).columns[0].lower()
    #         except:
    #             try:
    #                 stat_name = dataset_df.filter(regex='RURAL_URBAN_TYPE', axis=1).columns[0].lower()
    #             except:
    #                 try:
    #                     stat_name = dataset_df.filter(regex='COB_NAME', axis=1).columns[0].lower()
    #                 except:
    #                     stat_name = dataset_df.filter(regex='DEPCHILD_NAME', axis=1).columns[0].lower()

    last_record = offset

    for i in range(1000):
        print(f"trying records {last_record} - {last_record + record_limit}")
        api_url = f'/api/v01/dataset/{id}.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,{stat_name},obs_value&recordlimit={record_limit}&recordoffset={last_record}'
        # api_url = f'/api/v01/dataset/{id}.data.csv?measures=20100&select=date,geography_code,geography_type,geography_typecode,{stat_name},obs_value&recordlimit={record_limit}&recordoffset={last_record}'

        url = root_url + api_url

        print(url)

        try:
            batch_df = pd.read_csv(url)
        except pd.errors.EmptyDataError:
            print("empty dataset")
            break
        except:
            time.sleep(60)
            batch_df = pd.read_csv(url)

        display(batch_df.head())

        if len(batch_df) == 0:
            break

        dest_file = f'./data/raw/api/msoa/c2001/{mnemonic}_{i:02}.csv'
        batch_df.to_csv(dest_file)
        print(f"chunk {i} saved in {dest_file}")

        last_record += record_limit

downloading ks023
trying records 0 - 25000
http://www.nomisweb.co.uk/api/v01/dataset/NM_1631_1.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,commtype_name,obs_value&recordlimit=25000&recordoffset=0


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,COMMTYPE_NAME,OBS_VALUE
0,2001,E02002559,2001 super output areas - middle layer,305,All communal establishments,5
1,2001,E02002559,2001 super output areas - middle layer,305,Number of residents by communal establishment ...,34
2,2001,E02002559,2001 super output areas - middle layer,305,Medical and care establishments - NHS - Psychi...,0
3,2001,E02002559,2001 super output areas - middle layer,305,Medical and care establishments - NHS - Other ...,0
4,2001,E02002559,2001 super output areas - middle layer,305,Medical and care establishments - Local Author...,0


chunk 0 saved in ./data/raw/api/msoa/c2001/ks023_00.csv
trying records 25000 - 50000
http://www.nomisweb.co.uk/api/v01/dataset/NM_1631_1.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,commtype_name,obs_value&recordlimit=25000&recordoffset=25000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,COMMTYPE_NAME,OBS_VALUE
0,2001,E02002310,2001 super output areas - middle layer,305,Medical and care establishments - Other,0
1,2001,E02002310,2001 super output areas - middle layer,305,Other establishments,24
2,2001,E02002310,2001 super output areas - middle layer,305,Residents in establishments with limiting long...,71
3,2001,E02002310,2001 super output areas - middle layer,305,Residents in establishments with limiting long...,24
4,2001,E02002311,2001 super output areas - middle layer,305,All communal establishments,4


chunk 1 saved in ./data/raw/api/msoa/c2001/ks023_01.csv
trying records 50000 - 75000
http://www.nomisweb.co.uk/api/v01/dataset/NM_1631_1.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,commtype_name,obs_value&recordlimit=25000&recordoffset=50000


...

,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2001,E02002129,2001 super output areas - middle layer,305,All people,5903.0
1,2001,E02002129,2001 super output areas - middle layer,305,Males,2929.0
2,2001,E02002129,2001 super output areas - middle layer,305,Females,2974.0
3,2001,E02002129,2001 super output areas - middle layer,305,People living in households,5903.0
4,2001,E02002129,2001 super output areas - middle layer,305,People living in communal establishments,0.0


chunk 1 saved in ./data/raw/api/msoa/c2001/ks001_01.csv
trying records 50000 - 75000
http://www.nomisweb.co.uk/api/v01/dataset/NM_1634_1.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=50000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
0,2001,E02003199,2001 super output areas - middle layer,305,All people,9257.0
1,2001,E02003199,2001 super output areas - middle layer,305,Males,4458.0
2,2001,E02003199,2001 super output areas - middle layer,305,Females,4799.0
3,2001,E02003199,2001 super output areas - middle layer,305,People living in households,9239.0
4,2001,E02003199,2001 super output areas - middle layer,305,People living in communal establishments,18.0


chunk 2 saved in ./data/raw/api/msoa/c2001/ks001_02.csv
trying records 75000 - 100000
http://www.nomisweb.co.uk/api/v01/dataset/NM_1634_1.data.csv?geography=TYPE305&measures=20100&select=date,geography_code,geography_type,geography_typecode,cell_name,obs_value&recordlimit=25000&recordoffset=75000


,DATE,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,GEOGRAPHY_TYPECODE,CELL_NAME,OBS_VALUE
